# Utils

**Импорты библиотек:**

In [1]:
!pip install -q tqdm==4.66.5
!pip install -q torch==2.4.1+cu121
!pip install -q torchvision==0.19.1+cu121
!pip install -q transformers==4.44.2
!pip install -q comet_ml==3.47.6
!pip install -q triton==3.1.0
!pip install -q torchattacks==3.5.1
!pip install -q jax==0.4.38
!pip install -q jaxlib==0.4.38
!pip install -q seaborn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement torch==2.4.1+cu121 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==2.4.1+cu121

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.15.0
ERROR: Could not find a version that satisfies the requirement torchvision==0.19.1+cu121 (from versions: 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.17.2, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.20.0, 0.20.1)
ERROR: No matching distribution found for torchvision==0.19.1+cu121

[notice

In [3]:
import random

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import os

import comet_ml

import torch
import torchvision
from torchvision.transforms import v2
import torchattacks
from transformers import AutoImageProcessor, ViTModel, ViTConfig

**Utils:**

**Функция для фиксирования сида:**

In [4]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

**Additional utils:**

In [5]:
def get_root(env):
    # supports env == "colab", env == "kaggle"
    if env == "kaggle":
        return "/kaggle/working/"
    else:
        return "./"

In [6]:
# this augmentation first converts PIL image to Tensor,
# then casts resulting tensor to type=torch.float32,
# finally normalizes the image

default_aug = v2.Compose([v2.PILToTensor(),
                          v2.ToDtype(torch.float32, scale=True),
                          v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

In [7]:
# This function is used later in DataLoaders to fix the seed setting bugs when using several workers

def worker_init_fn(worker_id):
    torch_seed = torch.initial_seed() % (2**32)
    random.seed(torch_seed)
    np.random.seed(torch_seed)

**Датасет + DataLoaders:**

In [8]:
def get_CIFAR10_data(batch_size, train_transform=default_aug, test_transform=default_aug, env="colab"):
    root = get_root(env)
    data_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=train_transform, download=True)
    print("Prepare Train Set: ✅")
    data_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=test_transform)
    print("Prepare Test Set:  ✅")
    return data_train, data_test

In [9]:
def get_CIFAR10_dataloaders(batch_size, train_transform=default_aug, test_transform=default_aug, env="colab"):
    root = get_root(env)
    data_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=train_transform, download=True)
    dataloader_train = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=4, worker_init_fn=worker_init_fn)
    print("Prepare Train Set: ✅")
    data_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=test_transform)
    dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=4, worker_init_fn=worker_init_fn)
    print("Prepare Test Set:  ✅")
    return dataloader_train, dataloader_test

**Код для обычного/adversarial Fine-Tuning'а (на одном GPU):**

In [12]:
def eval_epoch(model, processor, loader, attack=None):
    val_loss = 0.
    val_acc = 0.

    model.eval()

    for data, target in tqdm(loader):
        #data = processor(data, return_tensors="pt")["pixel_values"]
        data = data.to(memory_format=torch.channels_last) # this line is for a speedup (used in a combination with amp)
        data = data.to(device)
        target = target.to(device)

        if attack is not None:
            # perform adversarial attack
            with torch.enable_grad():
                data = attack(data, target)
        
        logits = model(data)
        loss = torch.nn.functional.cross_entropy(logits, target)

        val_loss += (loss.item() * data.shape[0])
        val_acc += (torch.argmax(logits, dim=1) == target).sum().item()

    val_loss /= len(loader.dataset)
    val_acc /= len(loader.dataset)
    return val_loss, val_acc


def train_epoch(model, processor, optimizer, loader, scaler, epoch, exp, is_attack_used=False):
    train_loss = 0.
    train_acc = 0.

    step = (epoch - 1) * len(loader)

    model.train()

    for batch_idx, (data, target) in enumerate(tqdm(loader)):
        #data = processor(data, return_tensors="pt")["pixel_values"]
        data = data.to(memory_format=torch.channels_last) # this line is for a speedup (used in a combination with amp)
        data = data.to(device)
        target = target.to(device)

        if is_attack_used != "no":
            if is_attack_used == "BIM":
                attack = torchattacks.BIM(model, eps=(4 / 255), alpha=(2 / 255), steps=5)
            else:
                attack = torchattacks.PGD(model, eps=(4 / 255), alpha=(2 / 255), steps=5)
            # perform adversarial attack
            data = attack(data, target)

        optimizer.zero_grad()

        with torch.autocast("cuda", dtype=torch.float16, enabled=True):
            logits = model(data)
            loss = torch.nn.functional.cross_entropy(logits, target)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        with torch.no_grad():
            train_loss += loss.item() * data.shape[0]
            train_acc += (torch.argmax(logits, dim=1) == target).sum().item()

        if exp is not None:
            exp.log_metrics({"train loss (by steps)": loss.item()}, step=(step + batch_idx))

    train_loss /= len(loader.dataset)
    train_acc /= len(loader.dataset)
    return train_loss, train_acc, scaler


def train(model, processor, exp_name, optimizer, epochs, train_loader, val_loader, is_train_attack_used="no", scheduler=None, exp=None, env="colab"):
    best_val_acc = 0.
    best_val_acc_epoch = None
    best_model_state_filepath = None

    val_attacks_names = ["No Attack"]

    scaler = torch.GradScaler("cuda")

    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}:\n")
        metrics = dict()
        
        train_loss, train_acc, scaler = train_epoch(model, processor, optimizer, train_loader, scaler, epoch, exp, is_attack_used=is_train_attack_used)

        val_attacks = [None]
        
        for it in range(len(val_attacks)):
            val_loss, val_acc = eval_epoch(model, processor, val_loader, attack=val_attacks[it])
            metrics[val_attacks_names[it]] = {"Loss": val_loss, "Accuracy": val_acc}

        if scheduler is not None:
            scheduler.step()

        print("\n")
        print(" Train:")
        print(f"  Loss: {train_loss}, Accuracy: {train_acc}")
        print(" Validation:")
        for it in metrics.items():
            print(f"  Loss: {it[1]['Loss']}, Accuracy: {it[1]['Accuracy']}")


        if exp is not None:
            exp.log_metrics(
                   {"train loss": train_loss,
                    "train accuracy": train_acc},
                    step=(epoch * len(train_loader)))
            for it in metrics.items():
                exp.log_metrics(
                    {f"validation loss ({it[0]})": it[1]['Loss'],
                     f"validation accuracy ({it[0]})": it[1]['Accuracy']},
                     step=(epoch * len(train_loader)))

        val_acc = metrics["No Attack"]["Accuracy"]
        
        if val_acc > best_val_acc:
            print("🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉")
            best_val_acc = val_acc
            best_val_acc_epoch = epoch
            filepath = f"{get_root(env)}best_weights_{exp_name}_epoch{best_val_acc_epoch}.pth"
            torch.save(model.state_dict(), filepath)
            print("Save State of the Best Model: ✅")
            if best_model_state_filepath is not None:
                os.remove(best_model_state_filepath)
                print("Delete State File of the Previous Best Model: ✅")
            best_model_state_filepath = filepath

        print('-----------------------------------------------------------------\n')

    if exp is not None:
        exp.log_model(name=best_model_state_filepath[len(get_root(env)):-4],
                      file_or_folder=best_model_state_filepath)


def test(model, processor, loader, env="colab"):
    attacks_names = ["No Attack", "i-FGSM", "PGD", "DI-FGSM"]
    
    metrics = dict()

    attacks = [None, 
               torchattacks.BIM(model, eps=(8 / 255), alpha=(2 / 255), steps=10),
               torchattacks.PGD(model, eps=(8 / 255), alpha=(2 / 255), steps=10),
               torchattacks.DIFGSM(model, eps=(8 / 255), alpha=(2 / 255), steps=10, resize_rate=0.9, diversity_prob=0.5)
              ]
    
    for it in range(len(attacks)):
        loss, acc = eval_epoch(model, processor, loader, attack=attacks[it])
        metrics[attacks_names[it]] = {"Loss": loss, "Accuracy": acc}

    print("\n")
    print("Final Metrics:")
    for it in metrics.items():
        print(f" {it[0]}:")
        print(f"   Loss: {it[1]['Loss']}, Accuracy: {it[1]['Accuracy']}")


# Эксперименты

In [15]:
class ViTClassifier(torch.nn.Module):
    def __init__(self, encoder, num_classes):
        super(ViTClassifier, self).__init__()
        self.encoder = encoder
        
        head_hidden_dim = self.encoder.config.hidden_size * 2
        
        self.head = torch.nn.Sequential(
            torch.nn.Linear(self.encoder.config.hidden_size, head_hidden_dim),
            torch.nn.GELU(),
            torch.nn.Linear(head_hidden_dim, num_classes)
        )

    def forward(self, x):
        enc_features = self.encoder(x).last_hidden_state[:, 0, :]
        logits = self.head(enc_features)
        return logits

**ViT (SFT):**

In [16]:
comet_ml.login()

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)


Comet API key:  ········


COMET INFO: Valid Comet API Key saved in /home/mlcore/.comet.config (set COMET_CONFIG to change where it is saved).


In [17]:
# choose device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
NUM_EPOCHS = 10
BATCH_SIZE = 128
exp_name = "ViT_SFT"

In [19]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="colab")

# model
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224", use_fast=True)

vit_config = ViTConfig.from_pretrained("google/vit-base-patch16-224")
vit_config.image_size = 32
vit_config.patch_size = 8
encoder = ViTModel(vit_config)

model = ViTClassifier(encoder=encoder, num_classes=10).to(memory_format=torch.channels_last)
model.to(device)
#model = torch.compile(model)

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

100%|██████████| 170M/170M [02:48<00:00, 1.01MB/s]   


Extracting ./cifar-10-python.tar.gz to ./
Prepare Train Set: ✅
Prepare Test Set:  ✅


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

In [20]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, image_processor, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="no", scheduler=scheduler, exp=exp, env="colab")

exp.end()

print()
test(model, image_processor, test_loader, env="colab")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/ssl-adversarial1/5adcca5e063241dbb66cb86181d40bc2



Epoch 1:



COMET INFO: Couldn't find a Git repository in '/workdir' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
100%|██████████| 79/79 [00:01<00:00, 39.85it/s]




 Train:
  Loss: 1.761068134765625, Accuracy: 0.34798
 Validation:
  Loss: 1.5888404243469239, Accuracy: 0.4244
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:01<00:00, 44.11it/s]


...

100%|██████████| 79/79 [00:01<00:00, 44.71it/s]




 Train:
  Loss: 0.782916208782196, Accuracy: 0.72342
 Validation:
  Loss: 1.1482055152893067, Accuracy: 0.6015
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : ViT_SFT
COMET INFO:     url                   : https://www.comet.com/askoro/ssl-adversarial1/5adcca5e063241dbb66cb86181d40bc2
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [392]                           : (62155.625, 167467.96875)
COMET INFO:     train accuracy [10]                  : (0.34798, 0.72342)
COMET INFO:     train loss (by steps) [3910]         : (0.5504275560379028, 2.3515777587890625)
COMET INFO:     train loss [10]                      : (0.782916208782196, 1.761068134765625)
COMET INFO:     validation accuracy (No Attack) [10] : (0.4244, 0.6015)
COMET INFO:     validation loss (No Attack) [10]     : (1.14707779102325

100%|██████████| 79/79 [00:20<00:00,  3.87it/s]



Final Metrics:
 No Attack:
   Loss: 1.1482055152893067, Accuracy: 0.6015
 i-FGSM:
   Loss: 3.5661163051605222, Accuracy: 0.1626
 PGD:
   Loss: 5.683026221466064, Accuracy: 0.019
 DI-FGSM:
   Loss: 5.044743296051025, Accuracy: 0.0328


**ViT (Adversarial SFT with i-FGSM):**

In [21]:
comet_ml.login()

In [22]:
# choose device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
NUM_EPOCHS = 10
BATCH_SIZE = 128
exp_name = "ViT_Adversarial_SFT_BIM"

In [24]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="colab")

# model
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224", use_fast=True)

vit_config = ViTConfig.from_pretrained("google/vit-base-patch16-224")
vit_config.image_size = 32
vit_config.patch_size = 8
encoder = ViTModel(vit_config)

model = ViTClassifier(encoder=encoder, num_classes=10).to(memory_format=torch.channels_last)
model.to(device)
#model = torch.compile(model)

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [25]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, image_processor, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="BIM", scheduler=scheduler, exp=exp, env="colab")

exp.end()

print()
test(model, image_processor, test_loader, env="colab")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/ssl-adversarial1/b9d9e75eeb704c7aad3a1df967f6e5cc

COMET INFO: Couldn't find a Git repository in '/workdir' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:01<00:00, 43.48it/s]




 Train:
  Loss: 1.8706157482528687, Accuracy: 0.29718
 Validation:
  Loss: 1.5898978672027588, Accuracy: 0.4071
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:01<00:00, 44.00it/s]


...

100%|██████████| 79/79 [00:01<00:00, 44.59it/s]




 Train:
  Loss: 1.0668523699188233, Accuracy: 0.60824
 Validation:
  Loss: 1.0873052110671997, Accuracy: 0.6072
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : ViT_Adversarial_SFT_BIM
COMET INFO:     url                   : https://www.comet.com/askoro/ssl-adversarial1/b9d9e75eeb704c7aad3a1df967f6e5cc
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [392]                           : (74786.9375, 194678.6875)
COMET INFO:     train accuracy [10]                  : (0.29718, 0.60824)
COMET INFO:     train loss (by steps) [3910]         : (0.8551417589187622, 2.4318084716796875)
COMET INFO:     train loss [10]                      : (1.0668523699188233, 1.8706157482528687)
COMET INFO:     validation accuracy (No Attack) [10] : (0.4071, 0.6072)
COMET INFO:     validation loss (No Attack) [10]     :

100%|██████████| 79/79 [00:20<00:00,  3.87it/s]



Final Metrics:
 No Attack:
   Loss: 1.0873052110671997, Accuracy: 0.6072
 i-FGSM:
   Loss: 1.4469811721801757, Accuracy: 0.4686
 PGD:
   Loss: 2.7317657737731933, Accuracy: 0.1542
 DI-FGSM:
   Loss: 2.680039477157593, Accuracy: 0.1664


**ViT (Adversarial SFT with PGD):**

In [26]:
comet_ml.login()

In [27]:
# choose device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
NUM_EPOCHS = 10
BATCH_SIZE = 128
exp_name = "ViT_Adversarial_SFT_PGD"

In [29]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="colab")

# model
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224", use_fast=True)

vit_config = ViTConfig.from_pretrained("google/vit-base-patch16-224")
vit_config.image_size = 32
vit_config.patch_size = 8
encoder = ViTModel(vit_config)

model = ViTClassifier(encoder=encoder, num_classes=10).to(memory_format=torch.channels_last)
model.to(device)
#model = torch.compile(model)

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [30]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, image_processor, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="PGD", scheduler=scheduler, exp=exp, env="colab")

exp.end()

print()
test(model, image_processor, test_loader, env="colab")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/ssl-adversarial1/83881af3b9884e3b89b444ed50faed23

COMET INFO: Couldn't find a Git repository in '/workdir' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:01<00:00, 44.54it/s]




 Train:
  Loss: 2.1381961766815185, Accuracy: 0.17808
 Validation:
  Loss: 2.0734954469680784, Accuracy: 0.2002
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:01<00:00, 43.44it/s]


...

100%|██████████| 79/79 [00:01<00:00, 43.47it/s]




 Train:
  Loss: 1.9205758374786377, Accuracy: 0.29062
 Validation:
  Loss: 1.9685206230163574, Accuracy: 0.2603
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : ViT_Adversarial_SFT_PGD
COMET INFO:     url                   : https://www.comet.com/askoro/ssl-adversarial1/83881af3b9884e3b89b444ed50faed23
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [392]                           : (59401.125, 159423.0)
COMET INFO:     train accuracy [10]                  : (0.17808, 0.29062)
COMET INFO:     train loss (by steps) [3910]         : (1.7196159362792969, 2.9064483642578125)
COMET INFO:     train loss [10]                      : (1.9205758374786377, 2.1381961766815185)
COMET INFO:     validation accuracy (No Attack) [10] : (0.2002, 0.272)
COMET INFO:     validation loss (No Attack) [10]     : (1.9

100%|██████████| 79/79 [00:20<00:00,  3.87it/s]



Final Metrics:
 No Attack:
   Loss: 1.9685206230163574, Accuracy: 0.2603
 i-FGSM:
   Loss: 2.1599947494506835, Accuracy: 0.1821
 PGD:
   Loss: 2.03457209854126, Accuracy: 0.2423
 DI-FGSM:
   Loss: 2.011766353607178, Accuracy: 0.2532
